In [21]:
import os.path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import csv
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from utils.constants.dir_path import DataDirPath
from utils.constants.constants import Constants
from utils.convert_csv import ConvertCsv

In [22]:
dir_path = '/Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/for_angular/nnp-train_10_10/again'
# epoch = 15

base_info = DataDirPath.base_structure_info()

In [23]:
PATH = Constants.path()
base_df = pd.read_csv(f'{base_info}/base_info.csv')

In [24]:
# 各構造ごとのstructure_idxを全構造のindexに変更する
plus = 0
for structure in PATH:
    structure = structure.split('_')[-1]
    base_df.loc[base_df.structure == structure, ['structure_idx']] += plus
    idx = base_df[base_df.structure == structure].structure_idx.unique()
    idx.sort()
    plus = idx[-1]
    print(structure,idx[0],idx[-1])


alpha-critobalite 1 1150
alpha-quartz 1151 2250
beta-quartz 2251 3050
beta-trydymite 3051 4300
coesite 4301 5500
Fdd2-beta-critobalite 5501 6850
hex-trydymite 6851 7850
stishovite 7851 8550


In [27]:
# TODO epoch修正
def print_all_epoch_score(epoch):
    epoch = str(epoch).zfill(2)
    test_file = f'testpoints.0000{epoch}.out'
    train_file = f'trainpoints.0000{epoch}.out'

    if not os.path.exists(f'{dir_path}/analyze'):
        os.mkdir(f'{dir_path}/analyze')

    obj = ConvertCsv(output_dir=f'{dir_path}/analyze',target_dir=dir_path)
    obj.convert_n2p2_output_to_csv(10,test_file,'tmp_test')
    obj.convert_n2p2_output_to_csv(10,train_file,'tmp_train')

    def get_test_score(df_test_new):
        # テストに対して
        mae_test = mean_absolute_error(df_test_new['Eref'],df_test_new['Ennp'])
        mse_test = mean_squared_error(df_test_new['Eref'],df_test_new['Ennp'])
        rmse_test = np.sqrt(mse_test)
        r2_test = r2_score(df_test_new['Eref'],df_test_new['Ennp'])
        print('TEST')
        print(f'mae:{mae_test}, mse:{mse_test}, rmse:{rmse_test}, r2:{r2_test}')

    def get_train_score(df_train_new):
        # 訓練データに対して
        mae_train = mean_absolute_error(df_train_new['Eref'],df_train_new['Ennp'])
        mse_train = mean_squared_error(df_train_new['Eref'],df_train_new['Ennp'])
        rmse_train = np.sqrt(mse_train)
        r2_train = r2_score(df_train_new['Eref'],df_train_new['Ennp'])
        print('TRAIN')
        print(f'mae:{mae_train}, mse:{mse_train}, rmse:{rmse_train}, r2:{r2_train}')

    df_test = pd.read_csv(f'{dir_path}/analyze/tmp_test.csv')
    df_train = pd.read_csv(f'{dir_path}/analyze/tmp_train.csv')

    df_test_new = pd.merge(df_test,base_df,left_on='index',right_on='structure_idx')
    df_train_new = pd.merge(df_train,base_df,left_on='index',right_on='structure_idx')

    get_test_score(df_test_new)
    get_train_score(df_train_new)

    # structures =  base_df.structure.unique()
    # for structure in structures:
    #     print(structure)
    #     get_test_score(df_test_new[df_test_new['structure'] == structure])
    #     get_train_score(df_train_new[df_train_new['structure'] == structure])
for i in range(1,16):
    epoch = i
    print(f'epoch:{i}')
    print_all_epoch_score(epoch)

epoch:1
TEST
mae:0.003842558745476467, mse:2.423259274506665e-05, rmse:0.004922661144652012, r2:0.703894273333272
TRAIN
mae:0.005068331152849763, mse:0.005574132726734335, rmse:0.07466011469810595, r2:-69.43129162253618
epoch:2


KeyboardInterrupt: 

NameError: name 'df_test' is not defined